# Example Read Plan Printout Sections

### Imports

##### Standard Python Modules

In [1]:
from typing import Tuple
from pathlib import Path
from pprint import pprint
from functools import partial
import re

##### Public Modules from Anaconda

In [2]:
import pandas as pd

##### Sectionary Specific imports

In [3]:
import sections as sec
import text_reader as tr

## Text Processing Functions

### Split a text string into parts.
- The `delimiter=';'` argument tells it to split the string on ";"s.
- The `skipinitialspace=True` argument tells it to strip leading spaces from the
 text.

 For example:
 |Text|Becomes|
 |----|-------|
 |`'       Course;C1'`|`['Course', 'C1']`]|
 |`'Intent;1_PRIMARY'`|`['Intent', '1_PRIMARY']`]|
 |`'Plan Id;PELB FB'`|`['Plan Id', 'PELB FB']`]|
 |`'Technique;'`|`['Technique']`]|

In [4]:
dict_parse = tr.define_csv_parser(
    delimiter=';',
    skipinitialspace=True)

### Convert a list of two-item lists to a dictionary.
- First item in the sub-list is the key.  The second item is the value.
- `default_value=None` will cause one-item sub-lists to be dropped.

 For example the text:
 ```[
    ['Course', 'C1'],
    ['Intent', '1_PRIMARY'],
    ['Plan Id', 'PELB FB',
    ['Technique']
    ]```

becomes:
```{
    'Course': 'C1',
    'Intent': '1_PRIMARY',
    'Plan Id': 'PELB FB'
    }```

*Note: * `['Technique']` is dropped because it is a single-item list.

In [5]:
trim_dict = partial(tr.to_dict, default_value=None)

### Identify strings containing "Warning" text.
- a regular expression is used to identify the "Warning" text:
    - `'(?P<Num>[0-9]+)[. ]+'` Looks for one or more digits followed by a "." 
    and/or spaces.  This is assigned as the "Num" group.
    - `'WARNING[: ]*'`  The word "WARNING", followed by optional ":" 
    and/or spaces 
    - `'(?P<Warning>.*$)'`  The warning text is then the remainder of the 
    string. This is assigned as the "Warning" group.

If a match is found returns a *two*-item list: 
`["Num" group, "Warning" group]`.<br>
If a match is **not** found returns a *one*-item list: `[Original Text]`.

For example:
> `'1. WARNING: Plan target volume is different than plan primary reference
 point volume.'`

 Returns:
> `['1', 'Plan target volume is different than plan primary reference
 point volume.']`

And
> `'PhotonAlg; AAA_15606_Golden_Beam'`

Returns:
> `['PhotonAlg; AAA_15606_Golden_Beam']`


In [6]:
def get_warning(text_line):
    warning_pattern = re.compile(
        '(?P<Num>[0-9]+)'   # Warning index as Num group
        '[. ]+'             # delimiter and space
        'WARNING'           # warning text
        '[: ]*'             # delimiter and space
        '(?P<Warning>.*$)'  # remaining text in line as Warning group
        )
    warning_match = warning_pattern.search(text_line)
    if warning_match:
        indexer = warning_match.group('Num')
        warning_text = warning_match.group('Warning')
        warning_output = [f'Warning{indexer}', warning_text]
    else:
        warning_output = [text_line]
    return warning_output

### Parse the User origin text line.
- The text is expected to have the form of three numbers with 'cm' units 
contained in brackets.  For example:
> `(-1.26cm, 9.95cm, -4.70cm)`
- A regular expression is used to parse the "User Origin" text:
    - `'[^(]+.'` Everything up to and including the first bracket.
    - `'(?P<X>[0-9.-]+)'`  The X number group. 
    - `'[ cm,]*'`  'cm' units, spaces and comma 
    - `'(?P<Y>[0-9.-]+)'`  The Y number group.
    - `'[ cm,]*'`  'cm' units, spaces and comma 
    - `'(?P<Z>[0-9.-]+)'`  The Z number group.
    - `'[ cm,)]*'`  'cm' units, spaces, comma  and end bracket

If a match is found, returns four output items, each containing a 
two-item list:
> `[`<br>
    `['User Origin', `*The original text line after the '='*`],`<br>
    `['Origin X', `*The matched 'X' group*`],`<br>
    `['Origin Y', `*The matched 'Y' group*`],`<br>
    `['Origin Z', `*The matched 'Z' group*`]`<br>
    `]`

If a match is **not** found, returns a list containing the original text string 
split on ";"s.

For example:
> `'User Origin;User origin DICOM offset = (-1.26cm, 9.95cm, -4.70cm)'`

 Returns:
> `[`<br>
    `['User Origin', '(-1.26cm, 9.95cm, -4.70cm)'],`<br>
    `['Origin X', -1.26],`<br>
    `['Origin Y', 9.95],`<br>
    `['Origin Z', -4.70]`<br>
    `]`

And
> `'PhotonAlg; AAA_15606_Golden_Beam'`

Returns:
> `['PhotonAlg', 'AAA_15606_Golden_Beam']`




In [7]:
def get_origin(text_line):
    origin_pattern = re.compile(
        '[^(]+.'           # Everything up to and including the first bracket
        '(?P<X>[0-9.-]+)'  # X number group
        '[ cm,]*'          # Unit, space and comma
        '(?P<Y>[0-9.-]+)'  # Y number group
        '[ cm,]*'          # Unit, space and comma
        '(?P<Z>[0-9.-]+)'  # Z number group
        '[ cm,)]*'         # Unit, space, comma and end bracket
        )
    origin_match = origin_pattern.search(text_line)
    if origin_match:
        origin_str = text_line.split('=')[1].strip()
        origin = [
            ['User Origin', origin_str],
            ['Origin X', origin_match.group('X')],
            ['Origin Y', origin_match.group('Y')],
            ['Origin Z', origin_match.group('Z')]
            ]
    else:
        origin = [text_line.split(';')]
    for row in origin:
        yield row

### Parse the gantry text line.
- The text is expected to have the form of three numbers with 'cm' units 
contained in brackets. <br>
For example:
    - `Gantry;0.0 deg to - deg` 
    <br> or <br>
    - `Gantry;181.0 degCW to 179.0 deg`

- A regular expression is used to parse the gantry text:
    - `'(?P<GantryStart>[0-9.-]+)'` gantry start angle, assigned to 
    "GantryStart".
    - `'[ degtoCCW]*'`  Unit, space direction and "to" (not captured). 
    - `'(?P<GantryEnd>[0-9.-]+)'`  Gantry end angle, assigned to 
    "GantryEnd".
    - `'[ deg]*'`  'deg' units and space  (not captured).
    - `'[ cm,]*'`  'cm' units, spaces and comma 

If a match is found, returns either one or three output items, each containing a 
two-item list.<br>
> If *GantryEnd* contains `'-'`  (meaning gantry doesn't move), returns:<br>
    >> `[['Gantry', `*GantryStart*`]]`

> Otherwise (moving gantry), returns:<br>
    >> `[`<br>
    >> `['Gantry', `*GantryStart*`],`<br>
    >> `['GantryStart', `*GantryStart*`],`<br>
    >> `['GantryEnd', `*GantryEnd*`],`<br>
    >> `]`
                
If a match is **not** found returns a list containing the original text string 
split on ";"s.

For example:
> `Gantry;0.0 deg to - deg`

 Returns:
> `[['Gantry', '0.0']]`

Or
> `Gantry;181.0 degCW to 179.0 deg`

 Returns:
> `[`<br>
> `['Gantry', '181.0'],`<br>
> `['GantryStart', '181.0'],`<br>
> `['GantryEnd', '179.0'],`<br>
> `]`

And
> `'PhotonAlg; AAA_15606_Golden_Beam'`

Returns:
> `['PhotonAlg', 'AAA_15606_Golden_Beam']`

In [8]:
def get_gantry(text_line):
    gantry_pattern = re.compile(
        '(?P<GantryStart>[0-9.-]+)'  # gantry start group
        '[ degtoCCW]*'               # Unit, space direction and "to"
        '(?P<GantryEnd>[0-9.-]+)'    # gantry start group
        '[ deg]*'                    # Unit and space
        )
    gantry_match = gantry_pattern.search(text_line)
    if gantry_match:
        gantry_start = gantry_match.group('GantryStart')
        gantry_end = gantry_match.group('GantryEnd')
        if '-' in gantry_end:
            gantry = [
                ['Gantry', gantry_start]
                ]
        else:
            gantry = [
                ['Gantry', gantry_start],
                ['GantryStart', gantry_start],
                ['GantryEnd', gantry_end],
                ]
    else:
        gantry = [text_line.split(';')]
    for row in gantry:
        yield row

In [9]:
def clean_norm(text_line):
    if 'NO_ISQLAW_NORM' in text_line:
        norm_line = ['Norm Method', 'No Field Normalization']
    return norm_line

In [18]:
def drop_units(text: str) -> float:
    number_value_pattern = re.compile(
        # beginning of string and leading whitespace
        r'^\s*'                
        # value group contains optional initial sign and decimal place with 
        # number before and/or after.
        r'(?P<value>[-+]?\d+[.]?\d*)'
        # Optional whitespace between value and units
        r'\s*'              
                # value group contains optional initial sign and decimal place with 
        # number before and/or after.
        r'(?P<unit>[^\s]*)'        # beginning of value integer group
        r''           # units do not contain spaces
        r')'                # end of unit string group
        r'\s*'              # drop trailing whitespace
        r'$'                # end of string
        )

    find_num = number_value_pattern.search(text)
    if find_num:
        value, unit = find_num.groups()
        return value
    return text


def numeric_values(text_row: Tuple[str]) -> Tuple[str, float]:
    try:
        label, text_value = text_row
    except ValueError:
        return text_row
    numeric_value = drop_units(text_value)
    return (label, numeric_value)

## Define Plan Printout Sections

In [19]:
plan_section = sec.Section(
    start_section=None,
    end_section='PRESCRIPTION',
    processor=[tr.clean_ascii_text, dict_parse, tr.trim_items],
    aggregate=trim_dict,
    section_name='Plan')

prescription_section = sec.Section(
    start_section='PRESCRIPTION',
    end_section='IMAGE',
    processor=[tr.clean_ascii_text, dict_parse, tr.trim_items,
               numeric_values],
    aggregate=trim_dict,
    section_name='Prescription')

parse_origin = sec.Rule('User Origin', pass_method=get_origin)
image_parse = sec.RuleSet([parse_origin], default=dict_parse)
image_section = sec.Section(
    start_section='IMAGE',
    end_section='CALCULATIONS',
    processor=[tr.clean_ascii_text, image_parse, tr.trim_items,
               numeric_values],
    aggregate=trim_dict,
    section_name='Image')

parse_warning = sec.Rule('WARNING:', pass_method=get_warning)
calculation_parse = sec.RuleSet([parse_warning], default=dict_parse)
calculation_section = sec.Section(
    start_section='CALCULATIONS',
    end_section='WARNINGS',
    processor=[tr.clean_ascii_text, image_parse, tr.trim_items,
               numeric_values],
    aggregate=trim_dict,
    section_name='Calculations')

warning_section = sec.Section(
    start_section='WARNINGS',
    end_section='FIELDS DATA',
    processor=[tr.clean_ascii_text, calculation_parse, tr.trim_items,
               numeric_values],
    aggregate=trim_dict,
    section_name='Warnings')

parse_gantry = sec.Rule('G', pass_method=get_gantry, fail_method='Original')
no_norm = sec.Rule('NO_ISQLAW_NORM', pass_method=clean_norm)
field_parse = sec.RuleSet([parse_gantry, no_norm], default=dict_parse)
field_section = sec.Section(
    start_section=None,
    end_section=['END FIELD'],
    processor=[tr.clean_ascii_text, field_parse, tr.trim_items,
               numeric_values],
    aggregate=trim_dict,
    section_name='Field')

all_fields_section = sec.Section(
    start_section='FIELDS DATA',
    end_section='POINTS LOCATIONS',
    processor=[tr.clean_ascii_text],
    subsections=field_section,
    aggregate=tr.to_dataframe,
    section_name='Fields')

all_initial_sections = sec.Section(
    subsections=[plan_section, prescription_section, image_section,
                 calculation_section, warning_section],
    section_name='PlanCheck')

point_location_section = sec.Section(
    start_section=sec.SectionBreak('POINTS LOCATIONS', break_offset='after'),
    end_section=sec.SectionBreak('FIELD POINTS', break_offset='before'),
    processor=[tr.clean_ascii_text, dict_parse, tr.trim_items,
               numeric_values],
    aggregate=tr.to_dataframe,
    section_name='Point Locations')

point_dose_section = sec.Section(
    start_section=sec.SectionBreak('FIELD POINTS', break_offset='after'),
    end_section=sec.SectionBreak('PlanCheck', break_offset='before'),
    processor=[tr.clean_ascii_text, dict_parse, tr.trim_items,
               numeric_values],
    aggregate=tr.to_dataframe,
    section_name='Point Dose')



## Load the file as a list of strings.

In [20]:
#base_path = Path(r'\\dkphysicspv1\e$\Gregs_Work\Temp\Plan Checking Temp')
#test_file = base_path / PlanCheckText 2022-02-17 12-28-03.txt'

base_path = Path.cwd()   #  / 'examples'
test_file = base_path / 'PlanCheckText Test.txt'

test_text = test_file.read_text().splitlines()

In [21]:
plan_section.read(test_text)


{'PlanCheck': 'Test 1234567 TestPlan',
 'Patient Name': 'Patient, Test',
 'Patient Id': '01234567',
 'Date Of Birth': 'Jan 1, 1990',
 'Sex': 'Yes',
 'Primary Oncologist': 'Dr Bob',
 'Plan Status': 'Planning Approved',
 'Plan Approved On': 'January 1, 2022 12:00:00 AM',
 'Approved By': 'Carey Shenfield MD',
 'Checking Date': 'January 2, 2022 12:00:00 PM',
 'Checked By': 'Bo Derric',
 'Is Plan Modified?': '-',
 'Valid': 'All MU/Gy values are valid',
 'Course': 'C1',
 'Intent': '1_PRIMARY',
 'Plan Id': 'PELB FB',
 'Plan Name': 'PELB FB',
 'Plan Intent': 'Curative',
 'Technique': ''}

In [22]:
prescription_section.read(test_text)


error: unbalanced parenthesis at position 49

In [17]:
image_section.read(test_text)


{'ImageId': 'PELB FB',
 'Image Modality': 'CT',
 'ImageSeriesId': 'PELB FB 35687',
 'ImageSeriesComment': 'PROSTATE',
 'Contrast Exists': '-',
 'Imaging Device': 'Philips Big Bore',
 'User Origin': '(-1.26cm, 9.95cm, -4.70cm)',
 'Origin X': '-1.26',
 'Origin Y': '9.95',
 'Origin Z': '-4.70',
 'Imaging Orientation': 'Head First-Supine',
 'Treatment Orientation': 'Head First-Supine',
 'Coordinate System': 'Standard',
 'StructureSetId': 'PELB FB'}

In [18]:
calculation_section.read(test_text)


{'PhotonAlg': 'AAA_15606_Golden_Beam',
 'CalculationGridSizeInCM': '0.25',
 'CalculationGridSizeInCMForSRSAndHyperArc': '0.125',
 'FieldNormalizationType': '100% to isocenter',
 'HeterogeneityCorrection': 'ON',
 'Optimizer': 'PO_13623',
 'AirCavityCorrection': 'On',
 'InhomogeneityCorrection': 'On',
 'SmoothX': '40',
 'SmoothY': '30',
 'Calculation Medium': 'Inhomogeneity corrected',
 'Calculation Method': ''}

In [19]:
warning_section.read(test_text)


{'Warning1': 'Plan target volume is different than plan primary reference point volume.'}

In [20]:
field_section.read(test_text)

{'PlanCheck': 'Test 1234567 TestPlan',
 'Patient Name': 'Patient, Test',
 'Patient Id': '01234567',
 'Date Of Birth': 'Jan 1, 1990',
 'Sex': 'Yes',
 'Primary Oncologist': 'Dr Bob',
 'Plan Status': 'Planning Approved',
 'Plan Approved On': 'January 1, 2022 12:00:00 AM',
 'Approved By': 'Carey Shenfield MD',
 'Checking Date': 'January 2, 2022 12:00:00 PM',
 'Checked By': 'Bo Derric',
 'Is Plan Modified?': '-',
 'Valid': 'All MU/Gy values are valid',
 'Course': 'C1',
 'Intent': '1',
 'Plan Id': 'PELB FB',
 'Plan Name': 'PELB FB',
 'Plan Intent': 'Curative',
 'Technique': 'STATIC',
 'Patient Identifiers': '',
 'PatientId': '1234567',
 'Prescription': '',
 'Gantry': '0.0',
 'GantryStart': '1',
 'GantryEnd': '.',
 'Fractions': '25',
 'Start Delay': '-',
 'Fractions Per Week': '-',
 'Fractions Per Day': '-',
 'Normalization': '101.1',
 'Normalization Method': '100.00% covers 97.00% of Target Volume',
 'Warning': '',
 'Reference Point': '',
 'Primary Ref Point': 'PELB',
 'Primary Ref Point Rel

In [21]:
all_initial_sections.read(test_text)[0]


[{'PlanCheck': 'Test 1234567 TestPlan',
  'Patient Name': 'Patient, Test',
  'Patient Id': '01234567',
  'Date Of Birth': 'Jan 1, 1990',
  'Sex': 'Yes',
  'Primary Oncologist': 'Dr Bob',
  'Plan Status': 'Planning Approved',
  'Plan Approved On': 'January 1, 2022 12:00:00 AM',
  'Approved By': 'Carey Shenfield MD',
  'Checking Date': 'January 2, 2022 12:00:00 PM',
  'Checked By': 'Bo Derric',
  'Is Plan Modified?': '-',
  'Valid': 'All MU/Gy values are valid',
  'Course': 'C1',
  'Intent': '1_PRIMARY',
  'Plan Id': 'PELB FB',
  'Plan Name': 'PELB FB',
  'Plan Intent': 'Curative',
  'Technique': ''},
 {'Patient Identifiers': '',
  'Patient Name': 'Patient, Test',
  'PatientId': '1234567',
  'Date Of Birth': 'Jan 1, 1990',
  'Sex': 'Yes',
  'Primary Oncologist': 'Dr Bob',
  'Prescription': '',
  'Intent': '1',
  'Prescribed Dose': '4500.0',
  'Fractions': '25',
  'Start Delay': '-',
  'Fractions Per Week': '-',
  'Fractions Per Day': '-',
  'Dose Per Fraction': '180.0',
  'Normalization'

In [22]:
all_fields_section.read(test_text).T


,0,1,2,3,4
FieldId,KV AP PELB,KV RL PELB,KV LL PELB,CW,CCW
FieldName,KV AP PELB,,KV LL PELB,CW,CCW
Technique,STATIC,STATIC,STATIC,ARC,ARC
Linac,TR1,TR1,TR1,TR1,TR1
Gantry,0.0,270.0,90.0,327.3,327.3
Collimator,0.0,0.0,0.0,30.0,330.0
Couch,0.0,0.0,0.0,0.0,0.0
X,20.0,20.0,20.0,NaN,NaN
Y,20.0,20.0,20.0,NaN,NaN
Iso X,1.00,1.00,1.00,1.00,1.00


# Bug Alert:
Notice the duplicate first line!

In [23]:
point_location_section.read(test_text)


,Point,X,Y,Z
0,Point,X,Y,Z
1,PELB,-,-,-


In [24]:
point_dose_section.read(test_text)

,Field,Point,Dose,SSD,Depth,Effective Depth
0,Field,Point,Dose,SSD,Depth,Effective Depth
1,Plan,PELB,4500.0 cGy,,,
2,CW,PELB,89.0 cGy,-,-,-
3,CCW,PELB,91.0 cGy,-,-,-


In [ ]:
def patient_id(name, data):
    if isinstance(data, float):
        patient_id = '{:07.0f}'.format(data)
    elif isinstance(data, int):
        patient_id = '{:07d}'.format(data)
    else:
         patient_id = str(data).strip()
    return {name: patient_id}


def get_origin(name, data):
    origin_pattern = (
        '[^(]+.'          # Everything up to and including the first bracket
        '(?P<X>[0-9.-]+)'  # X number group
        '[ cm,]*'         # Unit, space and comma
        '(?P<Y>[0-9.-]+)'  # Y number group
        '[ cm,]*'         # Unit, space and comma
        '(?P<Z>[0-9.-]+)'  # Z number group
        '[ cm,)]*'        # Unit, space, comma and end bracket
        )
    origin_match = re.search(origin_pattern,data)
    if origin_match:
        origin_str = data.split('=')[1].strip()
        origin = {
            'User Origin': origin_str,
            'Origin X': origin_match.group('X'),
            'Origin Y': origin_match.group('Y'),
            'Origin Z': origin_match.group('Z')
            }
    else:
        origin = {}
    return origin


def get_gantry(name, data):
    gantry_pattern = (
        '(?P<GantryStart>[0-9.-]+)'  # gantry start group
        '[ degto]*'                  # Unit, space and "to"
        '(?P<GantryEnd>[0-9.-]+)'    # gantry start group
        '[ deg]*'                    # Unit and space
        )
    gantry_match = re.search(gantry_pattern,data)
    if gantry_match:
        gantry_start = gantry_match.group('GantryStart')
        gantry_end = gantry_match.group('GantryEnd')
        if gantry_end in '-':
            gantry = {
                'Gantry': gantry_start
                }
        else:
            gantry = {
                'Gantry': gantry_start,
                'GantryStart': gantry_start,
                'GantryEnd': gantry_end
                }
    else:
        gantry = {}
    return gantry


def clean_norm(name, data):
    if 'NO_ISQLAW_NORM' in data:
        data = 'No Field Normalization'
    return {name: data}



In [ ]:
def make_locations_table(printout_dict):
    def get_user_origin(printout_dict):
        image_dict = printout_dict['Image']
        user_origin = {
            'User Origin': {'X': image_dict['Origin X'],
                            'Y': image_dict['Origin Y'],
                            'Z': image_dict['Origin Z']
                            }
            }
        return user_origin

    def get_point_locations(printout_dict):
        point_locations = printout_dict['Point Locations']
        pt_idx = [''.join(['Point#', str(idx+1)])
                  for idx in range(len(point_locations))]
        pt_idx_series = pd.Series(pt_idx, name='Point Index')
        pt_locs = pd.concat([point_locations,pt_idx_series],axis='columns')
        pt_locs.set_index('Point Index', inplace=True)
        pt_locations = pt_locs.T.to_dict()
        return pt_locations

    def get_isocenter(printout_dict):
        field_data = printout_dict['Fields']
        fld1 = field_data.iloc[:,0]
        isoc = fld1.loc[['Iso X', 'Iso Y', 'Iso Z']]
        field_isocentre = {'Isocentre': {
            'X': isoc.at['Iso X'],
            'Y': isoc.at['Iso Y'],
            'Z': isoc.at['Iso Z']}
            }
        return field_isocentre

    locations = get_user_origin(printout_dict)
    locations.update(get_point_locations(printout_dict))
    locations.update(get_isocenter(printout_dict))

    locations_table = pd.DataFrame(locations).T
    return locations_table
def data_string(name, data):
    return {name: data}


In [ ]:
def printout_section_def():
    section_def = {
        'Plan': {
            'start': None,
            'end': 'PRESCRIPTION',
            'delimiter': ';',
            'read_method': 'Dictionary'},
        'Prescription': {
            'start': 'PRESCRIPTION',
            'end': 'IMAGE',
            'keep_start': False,
            'delimiter': ';',
            'read_method': 'Dictionary'},
        'Image': {
            'start': 'IMAGE',
            'end': 'CALCULATIONS',
            'keep_start': False,
            'delimiter': ';',
            'read_method': 'Dictionary'},
        'Calculations': {
            'start': 'CALCULATIONS',
            'end': 'WARNINGS',
            'keep_start': False,
            'delimiter': [';', ':'],
            'read_method': 'Dictionary'},
        # 'CalculationWarnings': {
        #     'start': 'CALCULATIONS',
        #     'end': 'WARNINGS',
        #     'keep_start': False,
        #     'delimiter': ':',
        #     'read_method': 'Dictionary'},
        'Warnings': {
            'start': 'WARNINGS',
            'end': 'FIELDS DATA',
            'keep_start': False,
            'delimiter': ';',
            'read_method': 'Dictionary'},
        'Fields': {
            'start': 'FIELDS DATA',
            'end': 'POINTS LOCATIONS',
            'keep_start': False,
            'delimiter': [';', ':'],
            'sub_sect_start': None,
            'sub_sect_end': 'END FIELD',
            'sub_sect_keep_start': True,
            'sub_sect_keep_end': False,
            'read_method': 'Multi Section'},
        'Point Locations': {
            'start': 'POINTS LOCATIONS',
            'end': 'FIELD POINTS',
            'keep_start': False,
            'delimiter': ';',
            'read_method': 'Table'},
        'Point Dose': {
            'start': 'FIELD POINTS',
            'end': None,
            'keep_start': False,
            'delimiter': ';',
            'read_method': 'Table'},
        }

    special_items = {
        'Patient Id': patient_id,
        'Normalization Method': rtd.data_string,
        'User Origin': get_origin,
        'FieldNormalizationType': rtd.data_string,
        'Norm Method': clean_norm,
        'Gantry': get_gantry,
        'BolusId': rtd.data_string
        }
    return section_def, special_items

def read_printout_file(file_path):
    # Load text
    text_rows = rtd.load_file(file_path)

    section_def, special_items = printout_section_def()
    printout_dict = rtd.load_sections(text_rows, section_def, special_items)
    printout_dict['Locations'] = make_locations_table(printout_dict)
    return printout_dict



In [ ]:
def main():
    '''Basic test code
    '''
    #%% select File
    data_path = Path.cwd()
    file_path = data_path / 'Test Files' / 'PlanCheck; Test1.txt'
    output_file_path = file_path = data_path / 'Test Files' / 'PlanCheckTest1.xlsx'
    # printout_dict
    new_printout_dict = read_printout_file(file_path)

    #%% save results
    new_printout_dict['Fields'] = new_printout_dict['Fields'].reset_index()
    new_printout_dict['Locations'] = new_printout_dict['Locations'].reset_index()
    output_book = st.open_book(output_file_path)
    for data_set_name, data in new_printout_dict.items():
        st.save_data_to_sheet(data, output_book, data_set_name,
                              starting_cell='H1', replace=False)


if __name__ == '__main__':
    main()
